In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.utils import prune
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW

dataset = load_dataset("NicolaiSivesind/human-vs-machine", "research_abstracts_labeled")['train'][0:10]
tokenizer = AutoTokenizer.from_pretrained("andreas122001/bloomz-3b-wiki-detector")
model = AutoModelForSequenceClassification.from_pretrained("andreas122001/bloomz-3b-wiki-detector")


# Load your dataset and preprocess it as needed
# Replace the following lines with your actual data loading and preprocessing code
texts = dataset['text']
labels = torch.tensor(dataset['label']) # Replace with your actual labels

# Tokenize and pad the input sequences
input_ids = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)['input_ids']

# Define your custom dataset and DataLoader
dataset = TensorDataset(input_ids, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define a pruning schedule
pruning_rate = 0.5
pruning_epochs = 1  # Number of epochs to prune the model

# Define the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training loop with pruning
for epoch in range(pruning_epochs):
    for input_batch, label_batch in dataloader:
        optimizer.zero_grad()
        outputs = model(input_batch.squeeze(), labels=label_batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Prune the model after each epoch
    prune.global_unstructured(
        parameters=model.classifier.parameters(),
        pruning_method=prune.L1Unstructured,
        amount=pruning_rate,
    )

# Save the pruned model
pruned_model_name = 'your-pruned-model-name'  # Replace with the desired name
model.save_pretrained(pruned_model_name)
tokenizer.save_pretrained(pruned_model_name)

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset human-vs-machine (/home/codespace/.cache/huggingface/datasets/NicolaiSivesind___human-vs-machine/research_abstracts_labeled/0.0.0/9e9ff0b78fa974ae55166fbed3b9032d432c39f2e76909d68bdd53cb4ea313de)
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]